In [1]:
import os
import torch

path = 'data'

# X0_1 = torch.load(f"{path}/X0_1.pt")
# X0_2 = torch.load(f"{path}/X0_2.pt")
# V_1 = torch.load(f"{path}/V_1.pt")
# V_2 = torch.load(f"{path}/V_2.pt")
# pos_1 = torch.load(f"{path}/pos_1.pt")
# pos_2 = torch.load(f"{path}/pos_2.pt")

# X0 = torch.cat((X0_1, X0_2), dim=0)
# V = torch.cat((V_1, V_2), dim=0)
# pos = torch.cat((pos_1, pos_2), dim=0)

# torch.save(X0, f"{path}/X0.pt")
# torch.save(V, f"{path}/V.pt")
# torch.save(pos, f"{path}/pos.pt")

X0, V, pos = torch.load(f"{path}/X0.pt"), torch.load(f"{path}/V.pt"), torch.load(f"{path}/pos.pt")
X0.shape


C:\Users\Sathya\AppData\Local\Temp\ipykernel_29120\24485433.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X0, V, pos = torch.load(f"{path}/X0.pt"), torch.load(f"{path

torch.Size([10000, 2])

In [2]:
from torus_math import Torus
torus = Torus(a=1, c=4)
pos_3d = torus.immersion(pos)
jacobian_matrices = torus.jacobian_matrix_batch(pos[:,0])
# jacobian_matrices.shape, V[:,0].unsqueeze(dim=-1).shape
V_3d = torch.bmm(jacobian_matrices, V[:,1].unsqueeze(dim=-1)).squeeze()
diff = pos_3d[:,1]-pos_3d[:,0]
diff, V_3d

(tensor([[-0.0915,  0.0097,  0.0339],
         [ 0.0339, -0.0234, -0.0708],
         [ 0.0050, -0.0734,  0.0109],
         ...,
         [ 0.0329,  0.0565, -0.0005],
         [ 0.0494,  0.0571, -0.0100],
         [-0.0005, -0.0324,  0.0037]], grad_fn=<SubBackward0>),
 tensor([[-0.0928,  0.0110,  0.0294],
         [ 0.0314, -0.0215, -0.0724],
         [ 0.0089, -0.0773,  0.0093],
         ...,
         [ 0.0318,  0.0547, -0.0004],
         [ 0.0484,  0.0554, -0.0099],
         [-0.0005, -0.0330,  0.0036]], grad_fn=<SqueezeBackward0>))

In [20]:
from torus_math import Torus
torus = Torus(a=1, c=4)
pos_3d = torus.immersion(pos[:,0])
jacobian_matrices = torus.jacobian_matrix_batch(X0)
jacobian_matrices.shape, V[:,0].unsqueeze(dim=-1).shape
V_3d = torch.bmm(jacobian_matrices, V[:,0].unsqueeze(dim=-1)).squeeze()
diff = pos_3d-torus.immersion(X0)
diff, V_3d

(tensor([[-0.0364,  0.0161,  0.0114],
         [-0.0262, -0.0295,  0.0349],
         [ 0.0242,  0.0933, -0.0192],
         ...,
         [ 0.0375, -0.0345,  0.0023],
         [ 0.0341,  0.0141,  0.0056],
         [ 0.0575, -0.0836, -0.0143]], grad_fn=<SubBackward0>),
 tensor([[-0.0367,  0.0156, -0.0106],
         [-0.0274, -0.0307, -0.0345],
         [ 0.0285,  0.0886,  0.0231],
         ...,
         [ 0.0377, -0.0342, -0.0036],
         [ 0.0346,  0.0143, -0.0057],
         [ 0.0546, -0.0775,  0.0149]], grad_fn=<SqueezeBackward0>))

In [6]:
i=2
(diff[:,i] - V_3d[:,i]).abs().median()

tensor(0.0004, grad_fn=<MedianBackward0>)

In [36]:
i=3
print(f'Euclidean displacement {pos[0].diff(dim=0)[i]}')
print(f'Velocities {V[0,i+1]}')

Euclidean displacement tensor([0.0049, 0.0557], grad_fn=<SelectBackward0>)
Velocities tensor([0.0054, 0.0557], grad_fn=<SelectBackward0>)


In [3]:
import torus_math
from geodesic_solver import Immersed_Manifold

torus = torus_math.Torus(a=1, c=4)
jm_torus_math = torus.jacobian_matrix_batch(X0)

manifold = Immersed_Manifold(immersion=torus.immersion)
jm_autograd = manifold.compute_partial_derivatives(X0).permute(dims=(0,2,1))

i=1
(jm_torus_math[i], jm_autograd[i])


(tensor([[-1.2316, -4.7206,  0.0000],
         [-0.4622,  0.1206,  0.8786]], grad_fn=<SelectBackward0>),
 tensor([[-1.2316, -0.4622],
         [-4.7206,  0.1206],
         [ 0.0000,  0.8786]], grad_fn=<SelectBackward0>))

In [7]:
jm_torus_math[0].shape, torch.linalg.pinv(jm_torus_math[0]).shape

(torch.Size([2, 3]), torch.Size([3, 2]))

In [5]:
import pickle
def unpickle_list(filename):
    """
    Unpickles a list from a file.

    Args:
        filename (str): The filename to load the pickled list from.

    Returns:
        list or None: The unpickled list, or None if an error occurred.
    """
    try:
        with open(filename, 'rb') as f:  # 'rb' for read binary
            loaded_list = pickle.load(f)
        print(f"List unpickled from {filename}")
        return loaded_list
    except FileNotFoundError:
        print(f"Error: File not found at {filename}")
        return None
    except (IOError, pickle.UnpicklingError) as e: #Handles more possible errors
        print(f"Error unpickling list: {e}")
        return None
    

unpickle_list('runtimes_dim_8.pkl')

List unpickled from runtimes_dim_8.pkl


[0.14872288703918457,
 0.09399938583374023,
 0.09020471572875977,
 0.08251452445983887,
 0.13127493858337402,
 0.09063410758972168,
 0.0845937728881836,
 0.10508394241333008,
 0.08269214630126953,
 0.09990859031677246]